In [9]:
%run ../Setup.ipynb

from isac.database.connection import database_connector as connector
from isac import configuration
from data_compare import sensor_data_provider as sdp

import pandas as pd
import math
import json
import base64

In [9]:
dataProvider = None

In [10]:
REQUEST = json.dumps({
    'path' : {},
    'args' : {
                'sensor_name': ['Simocode_Q11M1_Leistungen'],
                'wst': [1],
                'filter_by': ['eyJaZWl0Ijp7IiRndGUiOjE1NDUxMzMzMzY4OTAsIiRsdGUiOjE1NDUxMzUwMDAwMDB9fQ=='],
                'options': ['eyJub3JtYWxpemUiOnRydWV9']
             }
})

In [11]:
# Daten des Sensors Filtern
def filter_data(data, selected_wst):  
    cycle_every_n = 12 # Ein Zyklus ist nach n Signalen komplett
    
    # selected wst -1, da selected_wst von 1-6 statt 0-5
    wst_data = pd.concat([data[0 + (selected_wst-1) * 2::cycle_every_n],
                          data[1 + (selected_wst-1) * 2::cycle_every_n]]).sort_values(by=['Zeit'])

    return wst_data

In [12]:
# Wird für die JSON-Konvertierung benötigt
def handle_datatypes(self, o):
    if isinstance(o, numpy.int64): return int(o)  
    raise TypeError
        
def df_to_json(sensor_name, df):
    json_data = {}
    json_data['name'] = sensor_name
    json_data['data'] = df[['Zeit', 'Wert']].to_dict(orient = 'records')

    return json.dumps(json_data, default=handle_datatypes)

In [13]:
# GET /get_sensor_data
dataProvider = None
if dataProvider is None:
    dataProvider = sdp.SensorDataProvider()
    
req = json.loads(REQUEST)
args = req['args']

if 'sensor_name' not in args:
    print("Missing parameter: 'sensor_name'")
else:
    sensor_name = args['sensor_name'][0]
    filter_by = None
    options = None

    # MongoDB Data beim Auslesen direkt filtern
    if 'filter_by' in args:
        filter_by = json.loads(base64.b64decode(args['filter_by'][0]).decode("UTF-8"))

    data = dataProvider.getSensorData(sensor_name, filter_by)

    # Nach Werkstückträgern filtern
    if 'wst' in args:
        data = filter_data(data, int(args['wst'][0]))
    
    # Weitere Optionen
    if 'options' in args:
        options = json.loads(base64.b64decode(args['options'][0]).decode("UTF-8"))

        # Werte Y-Achse zwischen 0 und 1
        if 'normalize' in options and options['normalize'] == True:
            data["Wert"] = (data["Wert"] - min(data["Wert"])) / (max(data["Wert"]) - min(data["Wert"]))

    print(df_to_json(sensor_name, data))

{"name": "Simocode_Q11M1_Leistungen", "data": [{"Wert": 0.0, "Zeit": 1545133336890.0}, {"Wert": 1.0, "Zeit": 1545133725335.0}, {"Wert": 0.7525252525252525, "Zeit": 1545133729623.0}, {"Wert": 0.7424242424242424, "Zeit": 1545133730079.0}, {"Wert": 0.696969696969697, "Zeit": 1545133742947.0}, {"Wert": 0.6818181818181818, "Zeit": 1545133744499.0}, {"Wert": 0.6919191919191919, "Zeit": 1545133756183.0}, {"Wert": 0.6818181818181818, "Zeit": 1545133756547.0}, {"Wert": 0.6616161616161617, "Zeit": 1545133769052.0}, {"Wert": 0.6515151515151515, "Zeit": 1545133770240.0}, {"Wert": 0.6414141414141414, "Zeit": 1545133782740.0}, {"Wert": 0.6515151515151515, "Zeit": 1545133783564.0}, {"Wert": 0.6414141414141414, "Zeit": 1545133796797.0}, {"Wert": 0.6515151515151515, "Zeit": 1545133797984.0}, {"Wert": 0.6060606060606061, "Zeit": 1545133811673.0}, {"Wert": 0.6212121212121212, "Zeit": 1545133812405.0}, {"Wert": 0.5858585858585859, "Zeit": 1545133854206.0}, {"Wert": 0.5757575757575758, "Zeit": 154513385575